# Cooley-Turkey FFT Algorithm Notes

Compiled mostly from $\rightarrow$ https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm

The Cooley-Turkey Algorithm computes the Discrete Fourier Transform (DFT) in $\mathcal{O}(n\log n)$ time. The DFT is defined as 

$$X_k = \sum^{N-1}_{n=0} x_n e^{\frac{-2\pi i}{N} n k}$$

This transform produces a set of components $X_k$ in frequency space using a set of components $x_n$ in real space. To derive the Cooley-Turkey algorithm, we can start by splitting the discrete Fourier transform into a sum of even and odd indices. 

$$X_k = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N}(2m)k} + \sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N}(2m+1)k}$$

Multiplying through the parenthesis gives

$$X_k = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N/2}mk} + \sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N/2}mk+\frac{-2\pi i}{N}k}$$

Factoring out the second term we find that

$$X_k = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N/2}mk} + \sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N/2}mk}e^{\frac{-2\pi i}{N}k}$$

Since $e^{\frac{-2\pi i}{N}k}$ is not dependent on $m$ we can pull it out of the sum to produce the following

$$X_k = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N/2}mk} + e^{\frac{-2\pi i}{N}k} \sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N/2}mk}$$

Given our original equation for the DFT, we can see that the equation above is simply a weighted sum of the DFT'd even components $E_k$ and DFT'd odd components $O_k$ 

$$X_k = E_k + e^{-\frac{2\pi i }{N}k} O_{k}$$

Where 

$$E_k = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N/2}mk},\ \ \ O_{k} = \sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N/2}mk}$$

Furthermore, we only need to calculate the values of $E_k$ and $O_k$ for $k = 0, ... N/2 -1$. These intermediate values can then be used to compute the full set $X_k$ for $k=1, ... N$. This useful property is derived by first writing out the expression for the second half of the frequency component set.

$$X_{k+N/2} = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N/2}m(k+N/2)} + e^{\frac{-2\pi i}{N}(k+N/2)}\sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N/2}m(k+N/2)}$$

Multiplying through the parentheses yields

$$X_{k+N/2} = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N/2}mk-2\pi m i} + e^{\frac{-2\pi i}{N}k -\pi i}\sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N/2}mk-2\pi m i}$$

Factoring out the exponentials of sums produces

$$X_{k+N/2} = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N/2}mk}e^{-2\pi m i} + e^{\frac{-2\pi i}{N}k}e^{-\pi i}\sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N/2}mk}e^{-2\pi m i}$$

Given that $e^{-2\pi m i}=1$ for any $m$ and $e^{-\pi i} = -1$, we can write

$$X_{k+N/2} = \sum_{m=0}^{N/2-1}x_{2m}e^{\frac{-2\pi i}{N/2}mk} - e^{\frac{-2\pi i}{N}k}\sum_{m=0}^{N/2-1}x_{2m+1}e^{\frac{-2\pi i}{N/2}mk}$$

Using the definition for $E_k$ and $O_k$ defined above, we can see that

$$X_{k+N/2} = E_k - e^{\frac{-2\pi i}{N}k}O_k$$

Combining this with our work from earlier, our final algorithm reduces to a recursive formula for $X_k$ where $E_k$ and $O_k$ are the DFTs of the even and odd indexed sets of $x_n$. Here we only compute values for $k=0,...N/2$ to obtain the complete transform.

$$X_{k} = E_k + e^{\frac{-2\pi i}{N}k}O_k$$
$$X_{k+N/2} = E_k - e^{\frac{-2\pi i}{N}k}O_k$$

Using the original formula for the DFT, we can see that the base case of this recursive expression where $N=1$ is

$$X_0 = \sum^{1-1}_{n=0} x_n e^{\frac{-2\pi i}{1} n (0)}=x_0$$

This algorithm can be implemented in python as follows

In [43]:
import numpy as np
import cmath

In [44]:
def CooleyTurkeyRadix2Fft(x):
    """ Computes the unnormalized DFT of the input x using an out of place radix-2 Cooley Turkey Algorithm"""
    N = len(x)
    
    # --- base case ---
    if N == 1:
        return x
    
    # check to make sure the length of the input is a power of two
    if N%2 != 0:
        raise Exception("CooleyTurkeyRadix2FFT only accepts power of two input lengths")
    
    # --- recursion branches ---
    # DFT of even components Ek
    evenDft = CooleyTurkeyRadix2Fft(x[::2])
    # DFT of odd components Ok
    oddDft = CooleyTurkeyRadix2Fft(x[1::2])
    
    # allocate the return array
    returnArray = 1j * np.zeros(x.shape)
        
    # set the values in the return array
    for k in range(0, N//2):
        WeightedEk = evenDft[k]
        WeightedOk = np.exp(-2*np.pi*1j*k/N)*oddDft[k]
        
        returnArray[k] = WeightedEk + WeightedOk
        returnArray[k + N//2] = WeightedEk - WeightedOk
        
    return returnArray

As a simple test case, we can use a constant value of 1 for the input. Given that we have implemented the unnormalized transformation, the output should be {N, 0, 0 ...} (For a normalized transform, the output would instead be {1, 0, 0 ...})

In [45]:
x = np.ones(16)
print(CooleyTurkeyRadix2Fft(x))

[16.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j
  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]


For a more complicated test case, we can compare the output of CooleyTurkeyRadix2Fft with the built in numpy fft function to ensure our function behaves as expected

In [46]:
x = np.random.rand(2048) + 1j*np.random.rand(2048)

if np.allclose(CooleyTurkeyRadix2Fft(x), np.fft.fft(x)):
    print("passed!")
else:
    print("failed!")

passed!
